In [1]:
# Classes

class Card():
    '''A class to manage a single card'''
    
    def __init__(self,
                 value:str = None,
                 suit:str = None,
                 number:int = None):
        
        self.value = value
        self.suit = suit
        self.number = number
        self.name = self.value + ' of ' + self.suit
    
class Pile():
    '''A class to manage an ordered set of cards'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None):
        
        self.name = name
        
        if card_list is None:
            self.card_list = []
        else:
            self.card_list = card_list
        
    def shuffle(self):
        # This method is called to shuffle the pile
        
        random.shuffle(self.card_list)
                
    def pile_status(self):
        # This method will print the names of cards in the pile
        
            print('\n' + self.name)
            for card in self.card_list:
                print('  ' + card.name)
    
    def pile_size(self):
        # This method will print number of cards in the pile
        
            print('\n{pile} has {num} cards.'.format(pile = self.name, num = len(self.card_list)))
                
    def sort(self,
             suit_sort:bool = False):
        # This method will sort the cards in a pile ascending, by default value then suit
        
        if suit_sort is True:
            self.card_list.sort(key=lambda x: (x.suit, x.number))
        else:
            self.card_list.sort(key=lambda x: (x.number, x.suit))
        
class Hand(Pile):
    '''A class to manage a hand of cards, child class to Pile'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None,
                 owner:str = None):
        
        Pile.__init__(self,name,card_list)
        self.name = owner + "'s Hand"
        self.owner = owner
        
    def draw(self,
            pile:str = None,
            number:int = 1):
        # This method is called to draw a number of cards to the hand from the draw pile
        
        self.card_list.extend(pile.card_list[-number:])
        del pile.card_list[-number:]
        
    def find_book(self):
        # This method is called to identify a complete (4 cards) book of cards for the play_book method
        
        value_counter = {}
        
        for card in self.card_list:
            if card.value in value_counter:
                value_counter[card.value] += 1
            else:
                value_counter[card.value] = 1
            
            if value_counter[card.value] == 3:
                return card.value, [bookcard for bookcard in self.card_list if bookcard.value == card.value]
        
        return -1, []
        
    def play_book(self,
                  pile:str = None,
                  book:list = None):
        # This method is called to play a book of cards from the hand to the discard pile
        
        for card in book:
            self.card_list.remove(card)
            pile.card_list.append(card)
            
    def ask(self,
            value:str = None,
            target_hand:str = None):
        # This method is used to ask for a card from another player, "Do you have any X?"
        
        print('\n' + self.owner + ' asks ' + target_hand.owner + ' if they have any ' + value + '.')
        
        card_count = 0
        new_target_list = []
        for index, card in enumerate(target_hand.card_list.copy()):
            if card.value == value:
                self.card_list.append(card)
                card_count += 1
            else:
                new_target_list.append(card)
        target_hand.card_list = new_target_list
        self.sort()
        
        if card_count > 0:
            print('\n' + target_hand.owner + ' gives ' + str(card_count) + ' ' + value + ' to ' + self.owner + '.' )
        else:
            print('\n' + target_hand.owner + ' has no ' + value + '. ' + self.owner + ', go fish!')
        
    
class Scoreboard():
    '''A class to manage the overall game state'''
    
    def __init__(self,
                players:list = None,
                book_values:dict = None):
        
        if players is None:
            self.players = []
        else:
            self.players = players
        
        self.score = {}
        for player in self.players:
            self.score[player] = 0
        
        self.books = {}
        for key, value in book_values.items():
            self.books[value] = 0
            
    def update_score(self,
                    player:str = None,
                    value:str = None):
        # This method is called to update the score and denote a played book with value 1
        
        self.score[player] += 1
        self.books[value] = 1
    
    def print_score(self):
        # This method is called to print a summary of the game state
        
        print('\nScore  Player')
        for player in sorted(self.score, key = self.score.get, reverse = True):
            print(str(self.score[player]) + '      ' + player)
            
        played = [key for key, value in self.books.items() if value == 1]
        unplayed = [key for key, value in self.books.items() if value == 0]
        
        if len(played) > 0 and len(played) < 13:
            print('\n')
            print('Books in play:')
            for book in played:
                print(book)
        if len(unplayed) > 0:
            print('\n')
            print('Available books:')
            for book in unplayed:
                print('  ' + book)

In [10]:
# Game initialization
def start_gofish(player_list):
    # Set variables and check for valid player count
    cards, hands = [], []
    player_count = len(player_list)
        
    if player_count > 1 and player_count < 4:
        starting_hand_size = 7
    elif player_count > 3 and player_count < 7:
        starting_hand_size = 5
    else:
        return print('There must be between two and six players.')
    
    suits = {0:"Clubs", 1:"Diamonds", 2:"Hearts", 3:"Spades"}
    values = {0:"Ace", 1:"Two", 2:"Three", 3:"Four", 4:"Five", 5:"Six", 6:"Seven", 
              7:"Eight", 8:"Nine", 9:"Ten", 10:"Jack", 11:"Queen", 12:"King"}
    
    # Create the 52 playing cards
    for s in range(4):
        for v in range(13):
            cards.append(Card(value = values[v], suit = suits[s], number = v))
    
    # Create piles, scoreboards, and hands
    draw_pile = Pile(name = "Draw Pile", card_list = cards.copy())
    discard_pile = Pile(name = "Discard Pile")
    scoreboard = Scoreboard(players = player_list, book_values = values)
    for player in scoreboard.players:
        hands.append(Hand(owner = player))
    
    # Shuffle up and deal and sort hands
    draw_pile.shuffle()
    for hand in hands:
        hand.draw(pile = draw_pile, number = starting_hand_size)
        hand.sort()
    
    return cards, draw_pile, discard_pile, scoreboard, hands

def determine_player_count(playerlimit_lower, playerlimit_upper):
    # Determine and validate the count of players
    playercount = 0
    while playercount == 0:
        print('How many players ({lower}-{upper}) would like to play?'.format(lower = playerlimit_lower, upper = playerlimit_upper))
        userinput = input()
        if userinput.isdigit() and playerlimit_lower <= int(userinput) <= playerlimit_upper:
            playercount = int(userinput)
        else:
            print('Please enter a number of players between {lower} and {upper}.'.format(lower = playerlimit_lower, upper = playerlimit_upper)) 
    print("Great, starting a game for {count} players!".format(count = playercount))
    return playercount

def determine_playernames(playercount, char_limit = 16):
    ## MINOR BUG ## dupes can still occur with mixing upper and lower case characters
    # Determine, validate, and de-dupe the player names
    player_list = []
    for num in range(1, playercount + 1):
        playername = -1
        while playername == -1:
            print("What is Player {number}'s name?".format(number = num))
            userinput = input()[:char_limit] # limit the length of the name
            filteredinput = ''.join(filter(lambda x: x.isalnum(), userinput)) # filter out non alphanumeric characters
            if len(filteredinput) > 0:
                if len(userinput) != len(filteredinput):
                    print("Can I call you {name} instead? (Y/N)".format(name = filteredinput))
                    response = input()
                    if response in {'y','Y'}:
                        pass
                    else:
                        continue
            else:
                continue # player needs to have a non-empty name
            n = 0
            dedupedname = filteredinput
            while dedupedname in player_list: # players can't have the same name, not strictly necessary
                n += 1
                dedupedname = filteredinput + str(n)
            if dedupedname != filteredinput:
                print("Can I call you {name} instead? (Y/N)".format(name = dedupedname))
                response = input()
                if response in {'y','Y'}:
                    pass
                else:
                    continue
            playername = dedupedname
        print("Thank you, {name}!".format(name = playername))
        player_list.append(playername)
    return player_list

def start_turn_report(hands, turn, playerturn):
    # Print useful information for the player before they make a guess
    playerturn_name = hands[playerturn].name
    print("\n{name}'s Turn ({turn})".format(name = hands[playerturn].owner, turn = turn))
    hands[playerturn].pile_status()
    for hand in hands:
        if hand.name == playerturn_name:
            pass
        else:
            hand.pile_size()

def determine_card_options(hand):
    name_options = []
    num_options = []
    for card in hand.card_list:
        if card.value in name_options:
            pass
        else:
            name_options.append(card.value)
            num_options.append(str(card.number + 1))
    return name_options, num_options

def determine_player_options(hands, player_list, playerturn):
    name_options = []
    num_options = []
    for hand in hands:
        if hand.owner == player_list[playerturn]:
            continue
        elif len(hand.card_list) > 0:
            value = player_list.index(hand.owner) + 1
            name_options.append(hand.owner)
            num_options.append(str(value))
        else:
            pass
    return name_options, num_options

In [11]:
## Game start-up:
# First get input and validate number of players and their names with determine_player_count and determine_playernames
# Then initialize and deal out the cards with start_gofish
import string, random

playercount = determine_player_count(playerlimit_lower = 2, playerlimit_upper = 6)
player_list = determine_playernames(playercount, char_limit = 16)
cards, draw_pile, discard_pile, scoreboard, hands = start_gofish(player_list)

# Play the game until the endstate is reached -- all 13 books have been played
turn = 1
end_game = False
while end_game is False:
    end_turn = False # A player continues to play their turn if they guess a card correctly from another player OR draw pile
    while end_turn is False:
        playerturn = (turn - 1) % playercount # the index of the player whose turn it is
        start_turn_report(hands, turn, playerturn)
        
        player_name_options, player_num_options = determine_player_options(hands, player_list, playerturn)
        player_options = ["[{num}] {name}".format(num = num_option, name = name_option) for num_option, name_option in zip(player_num_options, player_name_options)]
        select_player = -1
        while select_player == -1:
            print('\nWhich player will you ask?\n   {players}'.format(players = "   ".join(player_options)))
            userinput = input()
            if userinput in player_name_options:
                select_player = userinput
            elif userinput in player_num_options:
                select_player = userinput
            else:
                continue
        
        card_name_options, card_num_options = determine_card_options(hands[playerturn])
        card_options = ["[{num}] {name}".format(num = num_option, name = name_option) for num_option, name_option in zip(card_num_options, card_name_options)]
        select_card = -1
        while select_card == -1:
            print('\nWhich card will you ask for?\n   {cards}'.format(cards = "   ".join(card_options)))
            userinput = input()
            if userinput in card_name_options:
                select_card = userinput
            elif userinput in card_num_options:
                select_card = userinput
            else:
                continue
        print(select_player, select_card)
        end_turn = True
    end_game = True


How many players (2-6) would like to play?
2
Great, starting a game for 2 players!
What is Player 1's name?
alex
Thank you, alex!
What is Player 2's name?
louis
Thank you, louis!

alex's Turn (1)

alex's Hand
  Two of Spades
  Three of Diamonds
  Five of Hearts
  Six of Diamonds
  Ten of Clubs
  Jack of Clubs
  Queen of Spades

louis's Hand has 7 cards.

Which player will you ask?
   [2] louis
2

Which card will you ask for?
   [2] Two   [3] Three   [5] Five   [6] Six   [10] Ten   [11] Jack   [12] Queen
2
2 2
